In [1]:
import pickle
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import glob
from tqdm import tqdm
from qdrant_client.models import PointStruct


client = QdrantClient(path="/home/ubuntu/alphapolitica/db") 
client.recreate_collection(
    collection_name="alphapolitica_face_db",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)

with open('data/filename.pickle', 'rb') as handle:
    facevecdb = pickle.load(handle)
def upload(id_, face_loc, face_encoding ):
    client.upsert(
    collection_name="alphapolitica_face_db",
    points=[
        PointStruct(
            id=id_,
            vector=face_encoding.tolist(),
            payload={"location": face_loc}
        )])
id_ = 0
for face_loc, face_encoding in tqdm(facevecdb.items()):
    face_loc_id = face_loc.split('faces/')[1].split('.png')[0].split('_face')[0]
    id_ =id_ + 1
    upload(id_, face_loc_id, face_encoding )
    if id_>100:
        break

In [1]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import glob
from tqdm import tqdm
first_arr_loaded = True
for face_loc in ['faces/xvIYQk-mDTU_2001_face_0.png']:#glob.glob('known_faces/*png'):
    print(face_loc)
    image_ = face_recognition.load_image_file(face_loc)
    face_locations = face_recognition.face_locations(image_)
    face_encoding = face_recognition.face_encodings(image_ , model= 'small')[0]
    if first_arr_loaded:
        known_faces_arr = np.reshape(face_encoding, (-1,len(face_encoding)))
        first_arr_loaded = False
    else:
        known_faces_arr = np.append(known_faces_arr,  np.reshape(face_encoding, (-1,len(face_encoding))), axis = 0) 

FileNotFoundError: [Errno 2] No such file or directory: 'data/filename.pickle'

In [ ]:
query_vector = face_encoding
hits = client.search(
    collection_name="alphapolitica_face_db",
    query_vector=query_vector,
    score_threshold=0.9  # Return 5 closest points
)